In [ ]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import smtplib
import re 

In [ ]:

def check_price():
    URL = 'https://www.amazon.in/dp/B0D3DWFY54/ref=va_live_carousel?pf_rd_r=YJ83VV32YS967FDMZK4S&pf_rd_p=418036cd-a3fa-48f1-a813-f80d4f52708d&pf_rd_m=A21TJRUUN4KGV&pf_rd_t=Gateway&pf_rd_i=desktop&pf_rd_s=desktop-6&pd_rd_w=hfFe8&pd_rd_r=ffedc8a7-13ef-41fd-9eef-d2d38cc580d1&pd_rd_wg=utxZz&linkCode=ilv&tag=onamzmaneetbh-21&ascsubtag=Ultra_HD_Savings_Top_4K_Smart_TVs_for_Every_Budget_250110143140&asc_contentid=amzn1.amazonlive.broadcast.a11471ad-1125-49bf-9642-410a55353b01&pd_rd_i=B0D3DWFY54&th=1&psc=1'

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    response = requests.get(URL, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch the page: {response.status_code}")
        return

    soup = BeautifulSoup(response.content, "html.parser")

    title_element = soup.find(id='productTitle')
    title = title_element.get_text(strip=True) if title_element else "Title not found"

    symbol = soup.find("span", class_="a-price-symbol")
    price_whole = soup.find("span", class_="a-price-whole")
    price_fraction = soup.find("span", class_="a-price-fraction")

    price_components = [
        symbol.get_text().strip() if symbol else '',
        price_whole.get_text().strip() if price_whole else '',
        price_fraction.get_text().strip() if price_fraction else ''
    ]
    price = ' '.join(price_components).strip()
    price = re.sub(r'[^\d.]', '', price)
    if '.' in price:
        price = price.split('.')[0]

    today = datetime.date.today()

    header = ['Title', 'Price', 'Date']
    data = [title, price, today]

    file_name = 'WebScraperDataset.csv'
    try:
        with open(file_name, 'a', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            if f.tell() == 0:  # Write header only if the file is empty
                writer.writerow(header)
            writer.writerow(data)

        print(f"Data written: {data}")
    except Exception as e:
        print(f"Error writing to CSV: {e}")

# Run the loop
try:
    while True:
        check_price()
        time.sleep(5)
except KeyboardInterrupt:
    print("Stopped by user")
